<a href="https://colab.research.google.com/github/pratikunterwegs/duke-squirrel/blob/master/landcover_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -U -q git+https://github.com/mccarthyryanc/folium_gee.git

In [10]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [44]:
# prelim code for earth engine landcover around duke

# get transects from user assets
downtown = ee.FeatureCollection("users/pratik_unterwegs/transect_squirrel_downtown")
forest = ee.FeatureCollection("users/pratik_unterwegs/transect_squirrel_forest")

# combine transects
transects = downtown.merge(forest)

# get a 1km buffer
buffer_size_m = 1000
transect_geom = transects.geometry()
transect_buffer = transect_geom.buffer(buffer_size_m)

# print information to check coords
long, lat = transect_buffer.getInfo()['coordinates']

# define start and end date
start_date = '2019-05-01'
end_date = '2019-10-01'

# filter sentinel by bounding box
# get sentinel data
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(transect_buffer)
sentinel_2019 = sentinel_bounds.filterDate(start_date, end_date)

# print some information
count = sentinel_2019.size().getInfo()
print("Count: ", count)

# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])

# map ndvi over sentinel
sentinel_ndvi = sentinel_2019.map(add_ndvi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').mean()

Count:  30


In [51]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#440154" quantity="-0" label="0"/>' + \
      '<ColorMapEntry color="#F46D43" quantity="0.1" label="100" />' + \
      '<ColorMapEntry color="#FDAE61" quantity="0.25" label="200" />' + \
      '<ColorMapEntry color="#FEE08B" quantity="0.35" label="300" />' + \
      '<ColorMapEntry color="#66BD63" quantity="0.45" label="400" />' + \
      '<ColorMapEntry color="#1A9850" quantity="0.55" label="500" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';

# print layers
vis = {'bands': ['ndvi']}
Map = emap.Map(center=[36,-79], zoom=10)
Map.addLayer(ndvi_median.sldStyle(sld_ramp), {}, 'ndvi')
# Map.addLayer(transect_buffer)
Map.addLayerControl()
Map